#### Milestone II
#### Michael Perrine
#### DSC 540
#### Professor Williams


# <h1><center>Market Analysis</center></h1>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

In [2]:
cea = pd.read_csv(r"coincident economic activity index.csv")
cea = pd.DataFrame(cea)
cea = cea.iloc[::-1].reset_index(drop=True)
cea.head(
    
)

,observation_date,USPHCI
0,2/1/2025,145.82
1,1/1/2025,145.61
2,12/1/2024,145.11
3,11/1/2024,144.65
4,10/1/2024,144.45


In [3]:
cli = pd.read_csv(r"composite leading indicators.csv")
cli = pd.DataFrame(cli)
cli = cli.iloc[::-1].reset_index(drop=True)
cli.head()


,observation_date,USALORSGPNOSTSAM
0,2023-11-01,100.4912
1,2023-10-01,100.4033
2,2023-09-01,100.3023
3,2023-08-01,100.1888
4,2023-07-01,100.0677


In [4]:
cpi = pd.read_csv(r"monthly cpi.csv")
cpi = pd.DataFrame(cpi)
cpi = cpi.iloc[::-1].reset_index(drop=True)
cpi.head()

,observation_date,CPIAUCSL
0,3/1/2025,319.615
1,2/1/2025,319.775
2,1/1/2025,319.086
3,12/1/2024,317.603
4,11/1/2024,316.449


In [5]:
mli = pd.read_csv(r"monthly leading indicators.csv")
mli = pd.DataFrame(mli)
mli = mli.iloc[::-1].reset_index(drop=True)
mli.head()

,observation_date,USSLIND
0,2/1/2020,1.72
1,1/1/2020,1.57
2,12/1/2019,1.48
3,11/1/2019,1.38
4,10/1/2019,1.41


In [6]:
t_bill = pd.read_csv(r"one year t-bill.csv")
t_bill = pd.DataFrame(t_bill)
t_bill = t_bill.iloc[::-1].reset_index(drop=True)
t_bill.head()

,observation_date,TB1YR
0,3/1/2025,3.90
1,2/1/2025,4.01
2,1/1/2025,4.00
3,12/1/2024,4.05
4,11/1/2024,4.14


In [7]:
dfs =[cea, cli, cpi, mli,t_bill]
dfs

[    observation_date  USPHCI
 0           2/1/2025  145.82
 1           1/1/2025  145.61
 2          12/1/2024  145.11
 3          11/1/2024  144.65
 4          10/1/2024  144.45
 ..               ...     ...
 549         5/1/1979   46.05
 550         4/1/1979   45.82
 551         3/1/1979   45.75
 552         2/1/1979   45.51
 553         1/1/1979   45.36
 
 [554 rows x 2 columns],
     observation_date  USALORSGPNOSTSAM
 0         2023-11-01          100.4912
 1         2023-10-01          100.4033
 2         2023-09-01          100.3023
 3         2023-08-01          100.1888
 4         2023-07-01          100.0677
 ..               ...               ...
 917       1947-06-01          100.8719
 918       1947-05-01          101.1850
 919       1947-04-01          101.5401
 920       1947-03-01          101.9178
 921       1947-02-01          102.3044
 
 [922 rows x 2 columns],
     observation_date  CPIAUCSL
 0           3/1/2025   319.615
 1           2/1/2025   319.775
 2        

In [8]:
new_data = reduce(lambda left, right: pd.merge(left, right, on = "observation_date", how = "outer"),dfs)


In [9]:
new_data = new_data.set_axis(["date", "cea", "cpi", "cli", "mli", "t_bill"], axis=1)
new_data

,date,cea,cpi,cli,mli,t_bill
0,1/1/1947,NaN,NaN,21.480,NaN,NaN
1,1/1/1948,NaN,NaN,23.680,NaN,NaN
2,1/1/1949,NaN,NaN,24.010,NaN,NaN
3,1/1/1950,NaN,NaN,23.510,NaN,NaN
4,1/1/1951,NaN,NaN,25.380,NaN,NaN
...,...,...,...,...,...,...
1856,9/1/2020,122.45,NaN,259.997,NaN,0.13
1857,9/1/2021,130.21,NaN,273.942,NaN,0.07
1858,9/1/2022,136.71,NaN,296.421,NaN,3.73
1859,9/1/2023,140.69,NaN,307.374,NaN,5.15


In [10]:
new_data = new_data.iloc[::-1].reset_index(drop=True)
new_data


,date,cea,cpi,cli,mli,t_bill
0,9/1/2024,144.20,NaN,314.851,NaN,3.87
1,9/1/2023,140.69,NaN,307.374,NaN,5.15
2,9/1/2022,136.71,NaN,296.421,NaN,3.73
3,9/1/2021,130.21,NaN,273.942,NaN,0.07
4,9/1/2020,122.45,NaN,259.997,NaN,0.13
...,...,...,...,...,...,...
1856,1/1/1951,NaN,NaN,25.380,NaN,NaN
1857,1/1/1950,NaN,NaN,23.510,NaN,NaN
1858,1/1/1949,NaN,NaN,24.010,NaN,NaN
1859,1/1/1948,NaN,NaN,23.680,NaN,NaN


In [11]:
new_data = new_data.replace(np.nan, 0)
new_data

,date,cea,cpi,cli,mli,t_bill
0,9/1/2024,144.20,0.0,314.851,0.0,3.87
1,9/1/2023,140.69,0.0,307.374,0.0,5.15
2,9/1/2022,136.71,0.0,296.421,0.0,3.73
3,9/1/2021,130.21,0.0,273.942,0.0,0.07
4,9/1/2020,122.45,0.0,259.997,0.0,0.13
...,...,...,...,...,...,...
1856,1/1/1951,0.00,0.0,25.380,0.0,0.00
1857,1/1/1950,0.00,0.0,23.510,0.0,0.00
1858,1/1/1949,0.00,0.0,24.010,0.0,0.00
1859,1/1/1948,0.00,0.0,23.680,0.0,0.00


In [12]:
new_data.dtypes

date       object
cea       float64
cpi       float64
cli       float64
mli       float64
t_bill    float64
dtype: object